# React Agent

In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

In [2]:
Settings.llm = Ollama(model = "qwen2.5:3b")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")